Given a string s, return the number of different non-empty palindromic subsequences in s. Since the answer may be very large, return it modulo 109 + 7.

A subsequence of a string is obtained by deleting zero or more characters from the string.

A sequence is palindromic if it is equal to the sequence reversed.

Two sequences a1, a2, ... and b1, b2, ... are different if there is some i for which ai != bi.

 

Example 1:

Input: s = "bccb"
Output: 6
Explanation: The 6 different non-empty palindromic subsequences are 'b', 'c', 'bb', 'cc', 'bcb', 'bccb'.
Note that 'bcb' is counted only once, even though it occurs twice.
Example 2:

Input: s = "abcdabcdabcdabcdabcdabcdabcdabcddcbadcbadcbadcbadcbadcbadcbadcba"
Output: 104860361
Explanation: There are 3104860382 different non-empty palindromic subsequences, which is 104860361 modulo 109 + 7.
 

Constraints:

1 <= s.length <= 1000
s[i] is either 'a', 'b', 'c', or 'd'.

In [ ]:
# brute force: take all the sub-sequences check that is palindrome or not and count the valid.
class Solution:
    def countPalindromicSubsequences(self, s: str) -> int:
        n = len(s)
        res = set()
        
        def recur(index, path):
            if index == n:
                if path == path[::-1] and path != "":
                    res.add(path)
                return
            
            # not take current character
            recur(index + 1, path)
            
            # take current character
            recur(index + 1, path + s[index])
        
        recur(0, "")
        return len(res)

# tc - O((2 ^ n) * n) generating all subsequences and checking palindrome.
# sc - O(2^n * n) for storing subsequences in the set.

## 🎯 **Case 2: When s[i] == s[j] - Three Subcases**

### **Subcase 2A: NO matching character in middle**
```
Example: s = "abcba", i=0, j=4
s[0] = s[4] = 'a', middle = "bcb" (no 'a')

Palindromes we can form:
• Single 'a'
• Double 'aa' (empty middle)  
• Wrapped: 'a' + middle_palindrome + 'a'

Middle has: ['b', 'c', 'bcb'] = 3 palindromes
Wrapped: ['aba', 'aca', 'abcba'] = 3 palindromes

Inside there is: 3
Ans wrappers can make another 3. So the ans for inside is 2 x  solve(i+1, j-1)

And there is a outer 2 'a' which can make the 'a' and 'aa' palindromes, so add 2.

So the finaly formula is:
Formula: 2 × solve(i+1, j-1) + 2
```

### **Subcase 2B: EXACTLY ONE matching character in middle**
```  
Example: s = "abaca", i=0, j=4
s[0] = s[4] = 'a', middle = "bac" (one 'a' at pos 2)

Palindromes we can form:
• Single 'a' (but don't double count with middle 'a')
• Double 'aa'
• Wrapped palindromes

Issue: Single 'a' = middle single 'a', so don't count twice!
Formula: 2 × solve(i+1, j-1) + 1
```

### **Subcase 2C: MULTIPLE matching characters in middle**
```
Example: s = "aabcaa", i=0, j=5  
s[0] = s[5] = 'a', middle = "abca"
First 'a' at pos 1, last 'a' at pos 4

Problem: Overcounting!
• Wrapping first 'a': a + 'a' + a = 'aaa'  
• Wrapping last 'a': a + 'a' + a = 'aaa'
Same palindrome counted twice!

Solution: Subtract the overcounted palindromes
Formula: 2 × solve(i+1, j-1) - solve(left+1, right-1)
```

## 🧠 **The Pattern:**

1. **Multiply by 2**: Each middle palindrome can be wrapped
2. **Add 2/1**: Account for single and double character palindromes  
3. **Subtract**: Remove overcounted duplicates when multiple occurrences exist

The recursion elegantly handles **all boundary cases** while avoiding **double counting**!

In [ ]:
class Solution:
    def countPalindromicSubsequences(self, s: str) -> int:
        MOD = 1000000007
        
        def solve(i, j):
            """
            Count distinct palindromic subsequences in s[i:j+1]
            
            Base cases:
            - If i > j: empty string → 0 palindromes
            - If i == j: single character → 1 palindrome
            """
            
            # Base cases
            if i > j:
                return 0
            if i == j:
                return 1
            
            # Case 1: First and last characters are different
            if s[i] != s[j]:
                # Include palindromes from s[i+1:j+1] and s[i:j]
                # But subtract overlap s[i+1:j] to avoid double counting.
                # solve for the 2 options and remove the solve(i + 1, j - 1) duplicate stuff.
                return (solve(i + 1, j) + solve(i, j - 1) - solve(i + 1, j - 1)) % MOD
            
            # Case 2: First and last characters are same (s[i] == s[j])
            else:
                char = s[i]
                
                # Find first and last occurrence of char inside [i+1, j-1]
                left = i + 1
                right = j - 1
                
                # Find leftmost occurrence of char in middle
                while left <= right and s[left] != char:
                    left += 1
                
                # Find rightmost occurrence of char in middle  
                while left <= right and s[right] != char:
                    right -= 1
                
                if left > right:
                    # No occurrence of char in middle
                    # We can form: single char + char+char + all middle palindromes wrapped
                    return (2 * solve(i + 1, j - 1) + 2) % MOD
                
                elif left == right:
                    # Exactly one occurrence of char in middle
                    # We get: single char + char+char + all middle palindromes wrapped
                    # But we don't double count the single char in middle
                    return (2 * solve(i + 1, j - 1) + 1) % MOD
                
                else:
                    # Multiple occurrences of char in middle
                    # We need to subtract the overcounted palindromes between first and last occurrence
                    return (2 * solve(i + 1, j - 1) - solve(left + 1, right - 1)) % MOD
        
        return solve(0, len(s) - 1)


# tc - O()

In [ ]:
class Solution:
    def countPalindromicSubsequences(self, s: str) -> int:
        MOD = 1000000007
        memo = {}
        
        def solve(i, j):
            """
            Count distinct palindromic subsequences in s[i:j+1]
            """
            
            # Check memoization first
            if (i, j) in memo:
                return memo[(i, j)]
            
            # Base cases
            if i > j:
                result = 0
            elif i == j:
                result = 1
            
            # Case 1: First and last characters are different
            elif s[i] != s[j]:
                # Include palindromes from both sides, subtract overlap
                result = (solve(i + 1, j) + solve(i, j - 1) - solve(i + 1, j - 1)) % MOD
            
            # Case 2: First and last characters are same (s[i] == s[j])
            else:
                char = s[i]
                
                # Find first and last occurrence of char inside [i+1, j-1]
                left = i + 1
                right = j - 1
                
                # Find leftmost occurrence of char in middle
                while left <= right and s[left] != char:
                    left += 1
                
                # Find rightmost occurrence of char in middle  
                while left <= right and s[right] != char:
                    right -= 1
                
                if left > right:
                    # No occurrence of char in middle
                    # We can form: single char + char+char + all middle palindromes wrapped
                    result = (2 * solve(i + 1, j - 1) + 2) % MOD
                
                elif left == right:
                    # Exactly one occurrence of char in middle
                    # We get: single char + char+char + all middle palindromes wrapped
                    # But we don't double count the single char in middle
                    result = (2 * solve(i + 1, j - 1) + 1) % MOD
                
                else:
                    # Multiple occurrences of char in middle
                    # We need to subtract the overcounted palindromes between first and last occurrence
                    result = (2 * solve(i + 1, j - 1) - solve(left + 1, right - 1)) % MOD
            
            # Store in memo and return
            memo[(i, j)] = result
            return result
        
        return solve(0, len(s) - 1)
    
# tc - Total states: O(n²) - all possible substring ranges
# With memo: O(n³) -> O(n²) states × O(n) work per state
# SC: O(n²) - memo table + O(n) recursion stack

In [ ]:
class Solution:
    def countPalindromicSubsequences(self, s: str) -> int:
        MOD = 1000000007
        n = len(s)
        
        # dp[i][j] = count of distinct palindromic subsequences in s[i:j+1]
        dp = [[0] * n for _ in range(n)]
        
        # Base case: single characters
        for i in range(n):
            dp[i][i] = 1
        
        # Fill table for all substring lengths
        for length in range(2, n + 1):  # length from 2 to n
            for i in range(n - length + 1):
                j = i + length - 1
                
                # Case 1: Different end characters
                if s[i] != s[j]:
                    dp[i][j] = (dp[i + 1][j] + dp[i][j - 1] - dp[i + 1][j - 1]) % MOD
                
                # Case 2: Same end characters
                else:
                    char = s[i]
                    
                    # Find first and last occurrence of char in middle [i+1, j-1]
                    left = i + 1
                    right = j - 1
                    
                    while left <= right and s[left] != char:
                        left += 1
                    while left <= right and s[right] != char:
                        right -= 1
                    
                    if left > right:
                        # Subcase 2A: No char in middle
                        dp[i][j] = (2 * dp[i + 1][j - 1] + 2) % MOD
                    
                    elif left == right:
                        # Subcase 2B: Exactly one char in middle
                        dp[i][j] = (2 * dp[i + 1][j - 1] + 1) % MOD
                    
                    else:
                        # Subcase 2C: Multiple chars in middle
                        dp[i][j] = (2 * dp[i + 1][j - 1] - dp[left + 1][right - 1]) % MOD
        
        return dp[0][n - 1]
    
# tc - Total states: O(n²) - all possible substring ranges
# With memo: O(n³) -> O(n²) states × O(n) work per state
# SC: O(n²) - memo table + O(n) recursion stack